In [ ]:
# se importa el conector de snowflake
import snowflake.connector 


In [ ]:
#se crea conexión
conn = snowflake.connector.connect(
    user='grupods03',
    password='Henry2022#',
    account='nr28668.sa-east-1.aws')

In [ ]:
def execute_query(connection, query):
    cursor = connection.cursor() # se inicializa la conexión Creates a cursor object. Each statement will be executed in a new cursor object.
    cursor.execute(query)
    cursor.close()

In [ ]:
query = "Create database DB_Pablo" # se inicializa en database prueba

execute_query(conn, query)

In [ ]:
sql = 'drop stage if exists data_stage'
execute_query(conn, sql)

sql = 'create stage data_stage file_format = (type = "csv" field_delimiter = "," skip_header = 1)'
execute_query(conn, sql)

In [ ]:
csv_file = '/Users/josetoledo/Desktop/prueba/df_hecho.csv'
sql = "PUT file://" + csv_file + " @DATA_STAGE auto_compress=true"
    
execute_query(conn, sql)

In [ ]:
import logging
from datetime import datetime, timedelta
import airflow
from airflow import DAG
from airflow.operators.python_operator import PythonOperator
from airflow.contrib.hooks.snowflake_hook import SnowflakeHook
from airflow.contrib.operators.snowflake_operator import SnowflakeOperator

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

args = {"owner": "Airflow", "start_date": datetime(2021,3,22,17,15)}

dag = DAG(
    dag_id="snowflake_connector3", default_args=args, schedule_interval=None
)

query1 = [
    """select 1;""",
    """show tables in database abcd_db;""",
]


def count1(**context):
    dwh_hook = SnowflakeHook(snowflake_conn_id="snowflake_conn")
    result = dwh_hook.get_first("select count(*) from abcd_db.public.test3")
    logging.info("Number of rows in `abcd_db.public.test3`  - %s", result[0])


with dag:
    query1_exec = SnowflakeOperator(
        task_id="snowfalke_task1",
        sql=query1,
        snowflake_conn_id="snowflake_conn",
    )

    count_query = PythonOperator(task_id="count_query", python_callable=count1)
query1_exec >> count_query
